In [6]:
cd eeg_matlab

C:\Users\whbom\projectspace\eeg_matlab


In [4]:


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from tqdm.notebook import tqdm
from tqdm import notebook, trange

import time
import pandas as pd


In [5]:
sti_number_or = ['1-back_mouth_revised', '1-back_nose_revised', '1-back_o2_revised', '2-back_mouth_revised', '2-back_nose_revised', '2-back_o2_revised', 'resting_closed_mouth_revised', 'resting_closed_nose_revised', 'resting_closed_o2_revised','resting_open_mouth_revised', 'resting_open_nose_revised', 'resting_open_o2_revised']
sti_number_ica = ['EOG_1-back_mouth_revised', 'EOG_1-back_nose_revised', 'EOG_1-back_o2_revised', 'EOG_2-back_mouth_revised', 'EOG_2-back_nose_revised', 'EOG_2-back_o2_revised', 'EOG_resting_closed_mouth_revised', 'EOG_resting_closed_nose_revised', 'EOG_resting_closed_o2_revised', 'EOG_resting_open_mouth_revised', 'EOG_resting_open_nose_revised', 'EOG_resting_open_o2_revised']

#sti_number_or =['resting_closed_nose_revised'] #WHAT IS or?
#sti_number_ica  = ['EOG_resting_closed_nose_revised'] #WHAT IS ica?

frequency_band=['delta', 'theta', 'alpha', 'beta', 'gamma'] 

'delta', 'theta', 'alpha', 'beta', 'gamma'

In [6]:
# sampling frequency
Fs = 250 
# sampling period
T = 1/Fs 
# channel number 
nCh = 35

# 각 sub band 범위
Wn = [0.5, 4, 8, 13, 30, 50]
sz = len(Wn)-1

In [7]:
order=['first','second','third','fourth']

In [8]:
freq_ratio = np.zeros((nCh,sz))
freq_ratio_abs = np.zeros((nCh,sz))

5분간격으로 fft를 사용해 data를 변환함
* 1분단위로 변환하기 위해서는 scan_durations parameter를 이용해 xi,L,t를 조정하면 될 것

In [12]:
data = np.loadtxt('./RawData/S1/EOG_1-back_nose_revised.txt') #shape:(35, 60000)

In [15]:
for iii in range(0,len(sti_number_ica)):
    for ii in range(0, 20):
        file_location = './RawData/S{}/{}.txt'.format(ii+1,sti_number_ica[iii])
        data_m = np.loadtxt(file_location)
        for tt in range(0,4):
            datat=data_m[:,int(tt*15000):int((tt+1)*15000)]

print(datat.shape)
print(data_m.shape)

(35, 15000)
(35, 60000)


In [48]:
int(4*(int(data.shape[1])/4))

NameError: name 'data' is not defined

In [12]:
int(t*(data.shape[1]/St)):int((t+1)*(int(data.shape[1])/St))

SyntaxError: illegal target for annotation (<ipython-input-12-8483188804bb>, line 1)

In [9]:
columns1=[]
for i in range(0,35):
        columns1.append('D{}'.format(i+1))
for k in range(0,35):
        columns1.append('T{}'.format(k+1))
for a in range(0,35):
        columns1.append('A{}'.format(a+1))
for b in range(0,35):
        columns1.append('B{}'.format(b+1))
for c in range(0,35):
        columns1.append('G{}'.format(c+1))

columns2=[]
for i in range(0,35):
    columns2.append('CH'+str(i+1))


In [ ]:
St=int(input("data split time : "))
for t in range(0,St):
    split_t0=int(t*(60000/St))
    split_t1=int((t+1)*(60000/St))

    print(data[:,split_t0:split_t1].shape)
    print(60000//St)

In [ ]:

    sub_num = int(input("subject :"))
    all_ratio = np.zeros((sub_num,nCh,sz)) #all_ratio_Relative
    all_ratio_abs = np.zeros((sub_num,nCh,sz))

    fin_all_ratio = []
    
    St=int(input("data split time : "))
    #data 1분씩 4개로 쪼개기
    for t in range(0,St):
        order_name=str(order[t])
        
        for iii in range(0,len(sti_number_ica)):
            txt_fname2=str(sti_number_ica[iii])

            for ii in range(0, sub_num):
                file_location = './RawData/S{}/{}.txt'.format(ii+1,sti_number_ica[iii])
                all_path ='./Tsplit_data/all/{}'.format(txt_fname2)

                #sti_number_ica 별 all subject fft data 저장 파일 생성 
                if not os.path.isdir(all_path):
                    os.makedirs(os.path.join(all_path)) 

                
                data_m = np.loadtxt(file_location)
                # data를 4분으로 쪼개 순차적으로 진행
                datat=data_m[:,int(t*(data.shape[1]/St)):int((t+1)*(int(data.shape[1])/St))]

                # 스캔시간
                scan_duration_s = datat.shape[1]/Fs
                # 0~scan_duration_s까지의 시간을 T steps, 실제 실험 시간
                #xi.shape = (60000,)
                xi = np.arange(0,scan_duration_s,T)
                # L=60000
                L = xi.shape[0]
                # t.shape = (60000,)
                t = np.arange(0,L)*T
                
                
                for i in range(0,nCh):

                    data1d_or = datat[i] #이게 멀까요?
                    data1d_ica = datat[i] #이것도요
                    if np.mean(data1d_or)==0:
                        data1d_ica = 0

                    data1d=data1d_ica

                    Y = np.fft.fft(data1d)

                    # Y/60000 절댓값 처리 이후 제곱, 대칭이룸
                    P2 = np.abs(Y/L)**2 
                    P1 = P2[0:int(L/2)+1]
                    # DC성분이기에 binary 경계로 인해 -1포함하지 않음
                    # 0을 빼고서는 negative power를 얻어내기 위해 *2
                    P1[1:-2] = 2*P1[1:-2] 
                    # P1(절반)의 실제 frequecy를 나타냄
                    f = Fs* (np.arange(0,int(L/2)+1))/L 

                    #가대역폭, bool형태
                    P1_valid = P1[(f>Wn[0])&(f<=Wn[-1])] 
                    f_valid = f[(f>Wn[0])&(f<=Wn[-1])]
                    # true count() 대체가능
                    total = sum(P1_valid) 

                    for isz in range(sz):
                        freq_ratio[i,isz] = sum(P1_valid[(f_valid>Wn[isz])&(f_valid<=Wn[isz+1])])/total*100 #relative power
                        freq_ratio_abs[i,isz] = sum(P1_valid[(f_valid>Wn[isz])&(f_valid<=Wn[isz+1])]) #abs, 개수


                        # subject parmeter값이 다수가 될 시 이 부분 수정필요
                        # 각 i열,isz 3D의 all ratio 1번째 행 따라 순차적으로 freq_ratio 값 대입 
                        all_ratio[ii,i,isz] = freq_ratio[i,isz]
                        # warning: all ratio[:,0,0]에도 feq_ratio값 대입되므로 non 처리
                        all_ratio[1:,:,:]=0
                        all_ratio_abs[ii,i,isz] = freq_ratio_abs[i,isz]
                        all_ratio_abs[1:,:,:]=0


                        fd_final= os.path.join('./Tsplit_data/{}_data_ratio_{}.txt'.format(order_name,txt_fname2))
                        final_save_fname = open(fd_final, 'w')
                        final_save_fname.write(str(freq_ratio))
                        final_save_fname.close()
                        #save(final_save_fname+'freq_ratio','-ascii','-double','-tabs'): 
                        # freq_ratio 변수를 만들고 16자리 정밀도를 가지는, 탭으로 구분된 텍스트 형식 파일에 저장

                        fin_all_ratio=all_ratio

                        for a in range(0, len(Wn)-1):
                            path='./Tsplit_data/all/'+txt_fname2+'/'+order_name+'_data'
                            if not os.path.isdir(path):
                                os.makedirs(os.path.join(path)) 
                            
                            #각 subband 따라 relative data file 생성
                            #save(all_ratio_save_name+'all_ratio2'+'-ascii'+'-double'+'-tabs')
                            all_ratio2 = str(all_ratio[:,:,a])
                            fd1=os.path.join(path,'all_ratio_{}.txt'.format(frequency_band[a]))
                            all_ratio_save_name = open(fd1,'w')
                            all_ratio_save_name.write(all_ratio2)
                            all_ratio_save_name.close()

                            #각 subband 따라 abs data file 생성
                            #save(all_ratio_ab_save_name+'all_ratio2_abs'+'-ascii'+'-double'+'-tabs')
                            all_ratio2_abs=str(all_ratio_abs[:,:,a])
                            fd2=os.path.join(path,'abs_all_ratio_{}.txt'.format(frequency_band[a]))
                            all_ratio_save_name = open(fd2,'w')
                            all_ratio_save_name.write(all_ratio2_abs)
                            all_ratio_save_name.close()

                            fin_all_ratio_1=str(fin_all_ratio[:,:,a])
                            fd3=os.path.join('./Tsplit_data/{}_Analysis_file_{}.txt'.format(order_name,frequency_band[a]))
                            fin_file_name=open(fd3,'w')
                            fin_file_name.write(fin_all_ratio_1)
                            fin_file_name.close()
                            #save(fin_file_name+'fin_all_ratio_1'+'-ascii'+'-double'+'-tabs')
    


In [60]:
def eeg_csv_save(sub_num, St):
    all_ratio = np.zeros((sub_num,nCh,sz)) #all_ratio_Relative
    all_ratio_abs = np.zeros((sub_num,nCh,sz))

    fin_all_ratio = []
    tt=0
    
    for n in notebook.tqdm(range(1000),desc='빨리좀 되라'):
        time.sleep(0.01)
        
        #data 1분씩 4개로 쪼개기
        for t in range(0,St):
            order_name=str(order[t])

            for iii in range(0,len(sti_number_ica)):
                txt_fname2=str(sti_number_ica[iii])

                print('2')

                for ii in range(0, sub_num):
                    file_location = './RawData/S{}/{}.txt'.format(ii+1,sti_number_ica[iii])
                    all_path ='./Tsplit_data/csv/{}'.format(txt_fname2)

                    #sti_number_ica 별 all subject fft data 저장 파일 생성 
                    if not os.path.isdir(all_path):
                        os.makedirs(os.path.join(all_path)) 

                    data_m = np.loadtxt(file_location)
                    # data를 4분으로 쪼개 순차적으로 진행
                    datat=data_m[:,int(tt*15000):int((tt+1)*15000)]

                    # 스캔시간
                    scan_duration_s = datat.shape[1]/Fs
                    # 0~scan_duration_s까지의 시간을 T steps, 실제 실험 시간
                    #xi.shape = (60000,)
                    xi = np.arange(0,scan_duration_s,T)
                    # L=60000
                    L = xi.shape[0]
                    # t.shape = (60000,)
                    t = np.arange(0,L)*T


                    for i in range(0,nCh):

                        data1d_or = datat[i] #이게 멀까요?
                        data1d_ica = datat[i] #이것도요
                        if np.mean(data1d_or)==0:
                            data1d_ica = 0

                        data1d=data1d_ica

                        Y = np.fft.fft(data1d)

                        # Y/60000 절댓값 처리 이후 제곱, 대칭이룸
                        P2 = np.abs(Y/L)**2 
                        P1 = P2[0:int(L/2)+1]
                        # DC성분이기에 binary 경계로 인해 -1포함하지 않음
                        # 0을 빼고서는 negative power를 얻어내기 위해 *2
                        P1[1:-2] = 2*P1[1:-2] 
                        # P1(절반)의 실제 frequecy를 나타냄
                        f = Fs* (np.arange(0,int(L/2)+1))/L 

                        #가대역폭, bool형태
                        P1_valid = P1[(f>Wn[0])&(f<=Wn[-1])] 
                        f_valid = f[(f>Wn[0])&(f<=Wn[-1])]
                        # true count() 대체가능
                        total = sum(P1_valid) 

                        
                        for isz in range(sz):
                            freq_ratio[i,isz] = sum(P1_valid[(f_valid>Wn[isz])&(f_valid<=Wn[isz+1])])/total*100 #relative power
                            freq_ratio_abs[i,isz] = sum(P1_valid[(f_valid>Wn[isz])&(f_valid<=Wn[isz+1])]) #abs, 개수


                            # subject parmeter값이 다수가 될 시 이 부분 수정필요
                            # 각 i열,isz 3D의 all ratio 1번째 행 따라 순차적으로 freq_ratio 값 대입 
                            all_ratio[ii,i,isz] = freq_ratio[i,isz]
                            all_ratio_abs[ii,i,isz] = freq_ratio_abs[i,isz]

                all_ratio_ravel=np.ravel(all_ratio, order='C')
                df_all_ratio_20by175=pd.DataFrame(all_ratio_ravel.reshape(20,175),columns=columns1)
                
                all_ratio_path=all_path+'/data_ratio_{}.csv'.format(order_name)
                df_all_ratio_20by175.to_csv(all_ratio_path,mode='w')
                #save(final_save_fname+'freq_ratio','-ascii','-double','-tabs'): 
                # freq_ratio 변수를 만들고 16자리 정밀도를 가지는, 탭으로 구분된 텍스트 형식 파일에 저장
                           
            if t.any()<=2:
                tt += 1
            else:
                continue
                
            print('tt={}'.format(tt))
    


In [61]:
eeg_csv_save(20,4)

2
2
2
2
2
2
2
2
2
2
2
2
tt=1
2
2
2
2
2
2
2
2
2
2
2
2
tt=2
2
2
2
2
2
2
2
2
2
2
2
2
tt=3
2
2
2
2
2
2
2
2
2
2
2
2
tt=4
2



C:\Users\whbom\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\whbom\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: Invalid number of FFT data points (0) specified.

mouse, nose, o2 = 0,1,2

In [11]:
import os
import glob

path = "./Tsplit_data/csv"
file_list = os.listdir(path)

for i in range(len(file_list)):
    csv_1_dir=path+'/'+file_list[i]
    csv_1_list=sorted(glob.glob(csv_1_dir+'/[!~]*.csv'))
    
print (csv_1_list)

['./Tsplit_data/csv/EOG_resting_open_o2_revised\\data_ratio_first.csv', './Tsplit_data/csv/EOG_resting_open_o2_revised\\data_ratio_fourth.csv', './Tsplit_data/csv/EOG_resting_open_o2_revised\\data_ratio_second.csv', './Tsplit_data/csv/EOG_resting_open_o2_revised\\data_ratio_third.csv']


In [63]:
path = "./Tsplit_data/csv"
file_list = os.listdir(path)
all_data=[]
    
for i in range(len(file_list)):
    csv_1_dir=path+'/'+file_list[i]
    csv_1_list=sorted(glob.glob(csv_1_dir+'/[!~]*.csv'))
        
    for file in csv_1_list:
        df=pd.read_csv(file)
        print(df.shape)
        all_data.append(df)
            
data_combine=pd.concat(all_data,axis=0)
data_combine

(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)
(20, 176)


,Unnamed: 0,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35
0,0,86.438244,2.939039,7.918901,2.125617,0.578198,92.180278,1.827242,4.263200,1.308323,...,95.532214,1.023974,1.950684,1.081038,0.412090,76.118354,7.418843,7.997842,6.510130,1.954831
1,1,61.492161,13.809933,6.063119,10.108544,8.526242,61.111762,13.925850,5.870033,10.638989,...,47.761585,11.132966,8.754775,19.254501,13.096173,42.053875,9.895022,12.748640,18.675124,16.627339
2,2,66.746460,9.554710,9.166596,11.041126,3.491108,67.040806,9.377440,8.762000,11.176861,...,82.613363,6.654056,4.078193,4.640252,2.014136,84.511366,3.670603,3.126598,6.276191,2.415243
3,3,61.054821,7.890001,6.892347,14.480890,9.681940,49.617741,11.112056,11.286982,16.653548,...,91.973985,2.311731,1.100666,2.649120,1.964498,54.244624,10.737137,16.261395,13.437727,5.319116
4,4,48.720676,15.231434,16.751631,15.032346,4.263913,51.344454,13.951978,16.725841,14.382267,...,71.547099,6.842257,8.897475,9.229514,3.483655,54.630718,10.429769,14.643907,19.614802,0.680805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,15,68.825055,11.272455,9.762631,7.108020,3.031840,74.056050,9.662509,6.825193,6.384445,...,78.590308,8.275332,4.989025,5.595196,2.550140,62.140511,10.939540,9.790407,11.427281,5.702261
16,16,52.717973,14.526718,10.372445,13.528484,8.854379,42.338968,9.979023,8.924494,23.523103,...,75.413882,5.686787,4.042026,8.865497,5.991809,61.968102,7.282525,6.683037,15.543054,8.523282
17,17,62.628063,18.440161,11.770133,5.756602,1.405041,68.388399,15.067572,9.440723,5.679031,...,82.450678,7.329485,4.726338,4.483873,1.009626,81.088808,6.683447,5.458840,5.347262,1.421643
18,18,76.391599,8.560106,5.467361,6.599574,2.981360,73.184865,9.208488,5.125282,8.238548,...,83.940503,6.697349,2.641460,4.328829,2.391859,70.164773,9.922144,6.941113,9.718117,3.253853


In [119]:
def dataset_time_split_csv(subject):
    
    #파일 불러들여서 list로 저장 
    path = "./Tsplit_data/csv"
    file_list = os.listdir(path)
    all_data=[]
    
    for i in range(len(file_list)):
        csv_1_dir=path+'/'+file_list[i]
        csv_1_list=sorted(glob.glob(csv_1_dir+'/[!~]*.csv'))
        
        for file in csv_1_list:
            df=pd.read_csv(file)
            all_data.append(df)
            
    data_combine=pd.concat(all_data,axis=0)
    data_combine=data_combine.drop(df.columns[[0]], axis='columns')

    data_combine["subject"]=np.nan
    data_combine["time"]=np.nan
    data_combine["target"]=np.nan
    data_combine["task"]=np.nan
    
    for s in range(48):
        for ss in range(10):
            data_combine["subject"][ss+20*s]="0"+str(int(ss+1))
        for sss in range(10,20):
            data_combine["subject"][sss+20*s]=str(int(sss+1))
    
    for t in range(12):
        data_combine["time"][(0+4*t)*subject:(1+4*t)*subject]="1"
        data_combine["time"][(1+4*t)*subject:(2+4*t)*subject]="4"
        data_combine["time"][(2+4*t)*subject:(3+4*t)*subject]="2"
        data_combine["time"][(3+4*t)*subject:(4+4*t)*subject]="3"
    
#1-back mouse, nose, o2
    data_combine["target"][0*subject:1*4*subject] = "010" 
    data_combine["target"][1*4*subject:2*4*subject] = "011" 
    data_combine["target"][2*4*subject:3*4*subject] = "012"
#2-back mouse, nose, o2
    data_combine["target"][3*4*subject:4*4*subject] = "110" #60:80
    data_combine["target"][4*4*subject:5*4*subject] = "111" #80:100
    data_combine["target"][5*4*subject:6*4*subject] = "112" #100:120
#rest close eye mouse, nose, o2
    data_combine["target"][6*4*subject:7*4*subject] = "200" #120:140
    data_combine["target"][7*4*subject:8*4*subject] = "201" #140:160
    data_combine["target"][8*4*subject:9*4*subject] = "202" #160:180
#rest open eye mouse, nose, o2
    data_combine["target"][9*4*subject:10*4*subject] = "210" #180:200
    data_combine["target"][10*4*subject:11*4*subject] = "211" #200:220
    data_combine["target"][11*4*subject:12*4*subject] = "212" #220:240

    #1-back mouse, nose, o2
    data_combine["task"][0*4*subject:1*4*subject] = "1back_m" #0:20
    data_combine["task"][1*4*subject:2*4*subject] = "1back_n" #21:40
    data_combine["task"][2*4*subject:3*4*subject] = "1back_O2" #41:60
    #2-back mouse, nose, o2
    data_combine["task"][3*4*subject:4*4*subject] = "2back_m" #60:80
    data_combine["task"][4*4*subject:5*4*subject] = "2back_n" #80:100
    data_combine["task"][5*4*subject:6*4*subject] = "2back_O2" #100:120
    #rest close eye mouse, nose, o2
    data_combine["task"][6*4*subject:7*4*subject] = "rest_close_m" #120:140
    data_combine["task"][7*4*subject:8*4*subject] = "rest_close_n" #140:160
    data_combine["task"][8*4*subject:9*4*subject] = "rest_close_O2" #160:180
    #rest open eye mouse, nose, o2
    data_combine["task"][9*4*subject:10*4*subject] = "rest_open_m" #180:200
    data_combine["task"][10*4*subject:11*4*subject] = "rest_open_n" #200:220
    data_combine["task"][11*4*subject:12*4*subject] = "rest_open_O2" #220:240
        
    data_combine.to_csv('./dataset_eeg_time_split.csv', index=False)
    
    return data_combine
    
    

In [120]:
dataset_time_split_csv(20)

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,G30,G31,G32,G33,G34,G35,subject,time,target,task
0,86.438244,2.939039,7.918901,2.125617,0.578198,92.180278,1.827242,4.263200,1.308323,0.420957,...,0.412090,76.118354,7.418843,7.997842,6.510130,1.954831,1.0,1,010,1back_m
1,61.492161,13.809933,6.063119,10.108544,8.526242,61.111762,13.925850,5.870033,10.638989,8.453366,...,13.096173,42.053875,9.895022,12.748640,18.675124,16.627339,2.0,1,010,1back_m
2,66.746460,9.554710,9.166596,11.041126,3.491108,67.040806,9.377440,8.762000,11.176861,3.642893,...,2.014136,84.511366,3.670603,3.126598,6.276191,2.415243,3.0,1,010,1back_m
3,61.054821,7.890001,6.892347,14.480890,9.681940,49.617741,11.112056,11.286982,16.653548,11.329674,...,1.964498,54.244624,10.737137,16.261395,13.437727,5.319116,4.0,1,010,1back_m
4,48.720676,15.231434,16.751631,15.032346,4.263913,51.344454,13.951978,16.725841,14.382267,3.595461,...,3.483655,54.630718,10.429769,14.643907,19.614802,0.680805,5.0,1,010,1back_m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,68.825055,11.272455,9.762631,7.108020,3.031840,74.056050,9.662509,6.825193,6.384445,3.071804,...,2.550140,62.140511,10.939540,9.790407,11.427281,5.702261,16.0,3,212,rest_open_O2
16,52.717973,14.526718,10.372445,13.528484,8.854379,42.338968,9.979023,8.924494,23.523103,15.234411,...,5.991809,61.968102,7.282525,6.683037,15.543054,8.523282,17.0,3,212,rest_open_O2
17,62.628063,18.440161,11.770133,5.756602,1.405041,68.388399,15.067572,9.440723,5.679031,1.424274,...,1.009626,81.088808,6.683447,5.458840,5.347262,1.421643,18.0,3,212,rest_open_O2
18,76.391599,8.560106,5.467361,6.599574,2.981360,73.184865,9.208488,5.125282,8.238548,4.242817,...,2.391859,70.164773,9.922144,6.941113,9.718117,3.253853,19.0,3,212,rest_open_O2


In [14]:
def dataset_time_split_csv_2(subject):
    
    #파일 불러들여서 list로 저장 
    path = "./Tsplit_data/csv"
    file_list = os.listdir(path)
    all_data=[]
    
    for i in range(len(file_list)):
        csv_1_dir=path+'/'+file_list[i]
        csv_1_list=sorted(glob.glob(csv_1_dir+'/[!~]*.csv'))
        
        for file in csv_1_list:
            df=pd.read_csv(file)
            all_data.append(df)
            
    data_combine=pd.concat(all_data,axis=0)
    data_combine=data_combine.drop(df.columns[[0]], axis='columns')

    data_combine["subject"]=np.nan
    data_combine["time"]=np.nan
    data_combine["target"]=np.nan
    data_combine["task"]=np.nan
    
    for s in range(48):
        for ss in range(10):
            data_combine["subject"][ss+20*s]="0"+str(int(ss+1))
        for sss in range(10,20):
            data_combine["subject"][sss+20*s]=str(int(sss+1))
    
    for t in range(12):
        data_combine["time"][(0+4*t)*subject:(1+4*t)*subject]="1"
        data_combine["time"][(1+4*t)*subject:(2+4*t)*subject]="4"
        data_combine["time"][(2+4*t)*subject:(3+4*t)*subject]="2"
        data_combine["time"][(3+4*t)*subject:(4+4*t)*subject]="3"
    
#1-back mouse, nose, o2
    data_combine["target"][0*subject:1*4*subject] = "0" 
    data_combine["target"][1*4*subject:2*4*subject] = "1" 
    data_combine["target"][2*4*subject:3*4*subject] = "0"
#2-back mouse, nose, o2
    data_combine["target"][3*4*subject:4*4*subject] = "0" #60:80
    data_combine["target"][4*4*subject:5*4*subject] = "1" #80:100
    data_combine["target"][5*4*subject:6*4*subject] = "0" #100:120
#rest close eye mouse, nose, o2
    data_combine["target"][6*4*subject:7*4*subject] = "0" #120:140
    data_combine["target"][7*4*subject:8*4*subject] = "1" #140:160
    data_combine["target"][8*4*subject:9*4*subject] = "0" #160:180
#rest open eye mouse, nose, o2
    data_combine["target"][9*4*subject:10*4*subject] = "0" #180:200
    data_combine["target"][10*4*subject:11*4*subject] = "1" #200:220
    data_combine["target"][11*4*subject:12*4*subject] = "0" #220:240

    #1-back mouse, nose, o2
    data_combine["task"][0*4*subject:1*4*subject] = "1back_m" #0:20
    data_combine["task"][1*4*subject:2*4*subject] = "1back_n" #21:40
    data_combine["task"][2*4*subject:3*4*subject] = "1back_O2" #41:60
    #2-back mouse, nose, o2
    data_combine["task"][3*4*subject:4*4*subject] = "2back_m" #60:80
    data_combine["task"][4*4*subject:5*4*subject] = "2back_n" #80:100
    data_combine["task"][5*4*subject:6*4*subject] = "2back_O2" #100:120
    #rest close eye mouse, nose, o2
    data_combine["task"][6*4*subject:7*4*subject] = "rest_close_m" #120:140
    data_combine["task"][7*4*subject:8*4*subject] = "rest_close_n" #140:160
    data_combine["task"][8*4*subject:9*4*subject] = "rest_close_O2" #160:180
    #rest open eye mouse, nose, o2
    data_combine["task"][9*4*subject:10*4*subject] = "rest_open_m" #180:200
    data_combine["task"][10*4*subject:11*4*subject] = "rest_open_n" #200:220
    data_combine["task"][11*4*subject:12*4*subject] = "rest_open_O2" #220:240
        
    data_combine.to_csv('./dataset_eeg_time_split_2.csv', index=False)
    
    return data_combine
    
    

In [15]:
dataset_time_split_csv_2(20)

C:\Users\whbom\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\whbom\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\whbom\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\whbom\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A valu

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,G30,G31,G32,G33,G34,G35,subject,time,target,task
0,86.438244,2.939039,7.918901,2.125617,0.578198,92.180278,1.827242,4.263200,1.308323,0.420957,...,0.412090,76.118354,7.418843,7.997842,6.510130,1.954831,1.0,1,0,1back_m
1,61.492161,13.809933,6.063119,10.108544,8.526242,61.111762,13.925850,5.870033,10.638989,8.453366,...,13.096173,42.053875,9.895022,12.748640,18.675124,16.627339,2.0,1,0,1back_m
2,66.746460,9.554710,9.166596,11.041126,3.491108,67.040806,9.377440,8.762000,11.176861,3.642893,...,2.014136,84.511366,3.670603,3.126598,6.276191,2.415243,3.0,1,0,1back_m
3,61.054821,7.890001,6.892347,14.480890,9.681940,49.617741,11.112056,11.286982,16.653548,11.329674,...,1.964498,54.244624,10.737137,16.261395,13.437727,5.319116,4.0,1,0,1back_m
4,48.720676,15.231434,16.751631,15.032346,4.263913,51.344454,13.951978,16.725841,14.382267,3.595461,...,3.483655,54.630718,10.429769,14.643907,19.614802,0.680805,5.0,1,0,1back_m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,68.825055,11.272455,9.762631,7.108020,3.031840,74.056050,9.662509,6.825193,6.384445,3.071804,...,2.550140,62.140511,10.939540,9.790407,11.427281,5.702261,16.0,3,0,rest_open_O2
16,52.717973,14.526718,10.372445,13.528484,8.854379,42.338968,9.979023,8.924494,23.523103,15.234411,...,5.991809,61.968102,7.282525,6.683037,15.543054,8.523282,17.0,3,0,rest_open_O2
17,62.628063,18.440161,11.770133,5.756602,1.405041,68.388399,15.067572,9.440723,5.679031,1.424274,...,1.009626,81.088808,6.683447,5.458840,5.347262,1.421643,18.0,3,0,rest_open_O2
18,76.391599,8.560106,5.467361,6.599574,2.981360,73.184865,9.208488,5.125282,8.238548,4.242817,...,2.391859,70.164773,9.922144,6.941113,9.718117,3.253853,19.0,3,0,rest_open_O2
